# SENTIMENT ANALYSIS ON TWITTER DATA

### IS MY GERMAN OR AMERICAN TWITTER NETWORK "HAPPIER" ?

# Step 0:
### Importing libraries and data:

In [164]:
import pandas as pd
import nltk
# nltk.download('punkt')
# nltk.download('wordnet')
# nltk.download('stopwords')
# nltk.download('vader_lexicon') 
from nltk.sentiment.vader import SentimentIntensityAnalyzer

import re
import string
import warnings
warnings.filterwarnings("ignore")
from textblob import TextBlob
from textblob_de import TextBlobDE
import matplotlib.pyplot as plt
import seaborn as sns
# sns.set_palette('Set3')
# sns.set_style("whitegrid")

In [165]:
# import df with tweets from previous notebook:
tweets = pd.read_csv('tweets_to_preprocess_NLP.csv')

In [166]:
tweets.drop('Unnamed: 0', axis=1, inplace=True)

In [167]:
tweets.drop('Geocode', axis=1, inplace=True)

In [169]:
tweets.drop(['Location_code', 'Friend', 'First_Name'], axis=1, inplace=True)

In [170]:
tweets.head()

,TweetText,Handle,ID,CreatedAt,Follower,Location,UserDescription,Country,Gender
0,RT @z_edian: Unser @snv_berlin Expert:innenwor...,christinacrupp,1359905846659059713,2021-02-11 16:43:14,Christina Rupp,"Bonn, Deutschland",Project Assistant International Cybersecurity ...,Germany,female
1,RT @rachel_groves: Daily goals: Wake up early....,joscchh,1359905846659059713,2021-02-11 16:43:14,Johanna Schäfer,"Bonn, Deutschland","CEO @BonnLAB, urban developer & #SocialMedia e...",Germany,female
2,RT @michael_adler_: Fing gut an! Bonn braucht ...,joscchh,1359101568390414337,2021-02-09 11:27:19,Johanna Schäfer,"Bonn, Deutschland","CEO @BonnLAB, urban developer & #SocialMedia e...",Germany,female
3,RT @EurAsiaBridgeEU: good topic - well done.,joscchh,1359019191358156802,2021-02-09 5:59:59,Johanna Schäfer,"Bonn, Deutschland","CEO @BonnLAB, urban developer & #SocialMedia e...",Germany,female
4,"Gestern haben @Lichtemomente, @askans, @michae...",joscchh,1359015354828210176,2021-02-09 5:44:44,Johanna Schäfer,"Bonn, Deutschland","CEO @BonnLAB, urban developer & #SocialMedia e...",Germany,female


# Step 1:
### Before pre-processing the data for NLP, separate tweet by language:

In [171]:
from langdetect import detect

In [172]:
detect(tweets['TweetText'][0])
# it detects the first tweet as being German

'de'

In [173]:
detect(tweets['TweetText'][1])
# and it detects the second tweet as being English. 

'en'

In [174]:
for index, row in tweets['TweetText'].iteritems():
    try:
        lang = detect(row)
        
    except:
        lang = 0
        
    tweets.loc[index, 'Language'] = lang

In [ ]:
# tweets['Language'].value_counts()

In [176]:
# tweets.drop(['Friend', 'Location_code'], axis=1, inplace=True)

In [177]:
# tweets.head()

### Basic Text Cleanup:

Remove RT, Punctuation, etc.

In [178]:
# add new column in df to clean up the texts
tweets['text'] = tweets['TweetText']

In [179]:
tweets['text']

0       RT @z_edian: Unser @snv_berlin Expert:innenwor...
1       RT @rachel_groves: Daily goals: Wake up early....
2       RT @michael_adler_: Fing gut an! Bonn braucht ...
3            RT @EurAsiaBridgeEU: good topic - well done.
4       Gestern haben @Lichtemomente, @askans, @michae...
                              ...                        
8420    RT @jana_hensel: Friedrich Merz hat eine Frau ...
8421    Ich hätte nie gedacht, dass ich das mal sage. ...
8422    RT @Mareicares: Dass da echt eine Frau ihren J...
8423    RT @Nilzenburger: Sich so einen perfiden Begri...
8424    Hat gerade ein Foto gepostet @ Berlin, Germany...
Name: text, Length: 8425, dtype: object

In [180]:
list_tweets = list(tweets['text'])

In [181]:
test= tweets.iloc[0][0]

In [182]:
test

'RT @z_edian: Unser @snv_berlin Expert:innenworkshop zur "staatlichen Beantwortung von böswilligen Cyberaktivitäten" geht langsam zuende.\n\nD…'

In [183]:
remove_rt = lambda x: re.sub('RT @\w+: ', "", x)

In [184]:
lab = lambda x: re.sub('[^äöüÄÖÜA-Za-z ]+','', x)

In [185]:
test1 = remove_rt(test)
test1

'Unser @snv_berlin Expert:innenworkshop zur "staatlichen Beantwortung von böswilligen Cyberaktivitäten" geht langsam zuende.\n\nD…'

In [186]:
test2 = lab(test1)

In [187]:
test2.lower()

'unser snvberlin expertinnenworkshop zur staatlichen beantwortung von böswilligen cyberaktivitäten geht langsam zuended'

In [188]:
# Make this into a function:

def clean_up(x):
    """ Clean up the text before NLP. 
    Remove Punctuation, remove retweet and @.
    Keep German characters as vowels."""
    
    remove_rt = lambda x: re.sub('RT @\w+: ', "", x)
    lab = lambda x: re.sub('[^äöüÄÖÜA-Za-z ]+','', x)
    tw_1 = remove_rt(x)
    tw_2 = lab(tw_1)
    tw_3 =  tw_2.lower()
    
    return tw_3
    

In [189]:
# Test the function with 1 tweet before applying to the column
clean_up('RT @z_edian: Unser @snv_berlin Expert:innenworkshop zur "staatlichen Beantwortung von böswilligen Cyberaktivitäten" geht langsam zuende.\n\nD…')

'unser snvberlin expertinnenworkshop zur staatlichen beantwortung von böswilligen cyberaktivitäten geht langsam zuended'

In [190]:
tweets['text'] = tweets['text'].apply(clean_up)

In [191]:
# tweets.head()

### Tokenization:

In [192]:
def tokenize(x):
    """Tokenize the tweets."""
    
    from nltk.tokenize import word_tokenize
    tokenized_x = word_tokenize(x)
    return tokenized_x

In [193]:
tokenize(tweets['text'][0])

['unser',
 'snvberlin',
 'expertinnenworkshop',
 'zur',
 'staatlichen',
 'beantwortung',
 'von',
 'böswilligen',
 'cyberaktivitäten',
 'geht',
 'langsam',
 'zuended']

In [194]:
tweets['tokenized_tw'] = tweets['text'].apply(tokenize)

In [195]:
tweets.shape

(8425, 12)

### Subset in German and English dataframes for the stemming/lemmatizing/vectorizing/analysis:

In [196]:
tweets_en = tweets[tweets['Language']=='en']
tweets_en.shape

(4791, 12)

In [197]:
tweets_de = tweets[tweets['Language']=='de']
tweets_de.shape

(3070, 12)

In [198]:
# So there are now a couple hundred tweets dropped from my analysis since the language was either not detected, or not detected as en/de.

### Stemming and Lemmatization:

In [199]:
def stem_lemmatize_en(l):
    """This function performs both stemming and lemmatization on our english tweets."""
    
    
    from nltk.stem import WordNetLemmatizer
    lemmatizer = WordNetLemmatizer()
    l = [lemmatizer.lemmatize(item) for item in l]
    from nltk.stem import SnowballStemmer
    stemmer = SnowballStemmer("english")
    l = [stemmer.stem(item) for item in l]
    
    return l 

In [200]:
tweets_en['stem_lemm_tw'] = tweets_en['tokenized_tw'].apply(stem_lemmatize_en)

In [201]:
# tweets_en.head()
# Does it actually look like the texts have changed enough from stemming and lemmatization?? 

In [202]:
def stem_lemmatize_de(l):
    """This function performs both stemming and lemmatization on our german tweets."""
    
    
    from nltk.stem import WordNetLemmatizer
    lemmatizer = WordNetLemmatizer()
    l = [lemmatizer.lemmatize(item) for item in l]
    from nltk.stem import SnowballStemmer
    stemmer = SnowballStemmer("german")
    l = [stemmer.stem(item) for item in l]
    
    return l 

In [203]:
tweets_de['stem_lemm_tw'] = tweets_de['tokenized_tw'].apply(stem_lemmatize_de)
# tweets_de.head()

### Stopwords Removal:

In [204]:
def remove_stopwords_en(l):
    """Remove english stopwords from a list of tweets."""
    
    from nltk.corpus import stopwords
    stopwords_list = stopwords.words("english")
    l = [item for item in l if not item in stopwords_list]
    
    return l

In [205]:
tweets_en['remove_stopwords_tw'] = tweets_en['stem_lemm_tw'].apply(remove_stopwords_en)
# tweets_en.head()

In [206]:
def remove_stopwords_de(l):
    """Remove german stopwords from a list of tweets."""
    
    from nltk.corpus import stopwords
    stopwords_list = stopwords.words("german")
    l = [item for item in l if not item in stopwords_list]
    
    return l

In [207]:
tweets_de['remove_stopwords_tw'] = tweets_de['stem_lemm_tw'].apply(remove_stopwords_de)
# tweets_de.head()

### Remove intermediary processing steps from dfs:

In [208]:
tweets_en.drop(['tokenized_tw', 'stem_lemm_tw', 'text', 'UserDescription', 'CreatedAt', 'ID', 'Follower'], axis=1, inplace=True)
tweets_en.rename(columns={'remove_stopwords_tw': 'text_processed'}, inplace=True)
tweets_en.head()

,TweetText,Handle,Location,Country,Gender,Language,text_processed
1,RT @rachel_groves: Daily goals: Wake up early....,joscchh,"Bonn, Deutschland",Germany,female,en,"[daili, goal, wake, earli, drink, coffe, work,..."
3,RT @EurAsiaBridgeEU: good topic - well done.,joscchh,"Bonn, Deutschland",Germany,female,en,"[good, topic, well, done]"
8,@jpbrice Life is always precious - every singl...,joscchh,"Bonn, Deutschland",Germany,female,en,"[jpbrice, life, alway, precious, everi, singl,..."
11,@PeterShepherd8 @MYorke27 @ZEReadyOregon Great...,joscchh,"Bonn, Deutschland",Germany,female,en,"[petershepherd, myork, zereadyoregon, great, l..."
12,RT @jpbrice: “You never know how strong you ar...,joscchh,"Bonn, Deutschland",Germany,female,en,"[never, know, strong, strong, onli, choic, quo..."


In [209]:
tweets_de.drop(['tokenized_tw', 'stem_lemm_tw', 'text', 'UserDescription', 'CreatedAt', 'ID', 'Follower'], axis=1, inplace=True)
tweets_de.rename(columns={'remove_stopwords_tw': 'text_processed'}, inplace=True)
tweets_de.head()

,TweetText,Handle,Location,Country,Gender,Language,text_processed
0,RT @z_edian: Unser @snv_berlin Expert:innenwor...,christinacrupp,"Bonn, Deutschland",Germany,female,de,"[snvberlin, expertinnenworkshop, staatlich, be..."
2,RT @michael_adler_: Fing gut an! Bonn braucht ...,joscchh,"Bonn, Deutschland",Germany,female,de,"[fing, gut, bonn, braucht, mindshift, autostad..."
4,"Gestern haben @Lichtemomente, @askans, @michae...",joscchh,"Bonn, Deutschland",Germany,female,de,"[gest, lichtemoment, askan, michaeladl, barbar..."
5,@VolkerGoebbels Macht Sinn 😏,joscchh,"Bonn, Deutschland",Germany,female,de,"[volkergoebbel, macht, sinn]"
6,@jbrunotte Was waren denn deine Beweggründe da...,joscchh,"Bonn, Deutschland",Germany,female,de,"[jbrunott, wa, beweggrund, dafur, spannend, wiss]"


# Step 2: 
### Sentiment Analysis with TextBlob

Calculating Negative, Positive, and/or Neutral Values.
* REMEMBER - the tweets in tweets_en means that they are English-language tweets, NOT that they come from my American network. 
* REMEMBER - the tweets in tweets_de means that they are German-language tweets, NOT that they come from my German network.

#### Using Python package TextBlob to calculate polarity values of individual tweets:

In [210]:
sentiment_objects_en = [TextBlob(tweet) for tweet in tweets_en['TweetText']]

sentiment_objects_en[0].polarity, sentiment_objects_en[0]

(0.19404761904761905,
 TextBlob("RT @rachel_groves: Daily goals: Wake up early. Drink coffee. Work hard. Be ambitious. Do what you love. Love what you do. Have fun. #inspir…"))

In [211]:
# Add polarity column to the dataframes:

tweets_en['Polarity'] = [i.polarity for i in sentiment_objects_en]
# tweets_en.head()

In [212]:
sentiment_objects_de = [TextBlobDE(tweet) for tweet in tweets_de['TweetText']]

sentiment_objects_de[0].polarity, sentiment_objects_de[0]

(-0.35,
 TextBlobDE("RT @z_edian: Unser @snv_berlin Expert:innenworkshop zur "staatlichen Beantwortung von böswilligen Cyberaktivitäten" geht langsam zuende.
 
 D…"))

In [213]:
tweets_de['Polarity'] = [i.polarity for i in sentiment_objects_de]
# tweets_de.head()

### Add polarity column to df of all tweets and plot polarity by location rather than language.

#### Using Python package TextBlob to calculate sentiment (pos/neg) values of individual tweets:

TextBlob sentiment returns polarity score and subjectivity score. 
If polarity > 0, it is considered positive, if  polarity < 0, it is considered negative.

In [214]:
sentiment_objects_en = [TextBlob(tweet) for tweet in tweets_en['TweetText']]

sentiment_objects_en[0].sentiment, sentiment_objects_en[0]

(Sentiment(polarity=0.19404761904761905, subjectivity=0.42738095238095236),
 TextBlob("RT @rachel_groves: Daily goals: Wake up early. Drink coffee. Work hard. Be ambitious. Do what you love. Love what you do. Have fun. #inspir…"))

In [215]:
## Add column w. subjectivity score to each df
tweets_en['Subjectivity'] = [i.sentiment[1] for i in sentiment_objects_en]
# tweets_en.head()

In [216]:
tweets_de['Subjectivity'] = [i.sentiment[1] for i in sentiment_objects_de]
# tweets_de.head()

#### Add column with sentiment (pos/neg/neu)

In [217]:
sentiment_category_en = ['positive' if score >0
                         else 'negative' if score<0
                             else 'neutral'
                                  for score in tweets_en['Polarity']]

In [218]:
sentiment_category_en[:20] 
# Make sure it worked, then add to df

['positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'neutral',
 'neutral',
 'neutral',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'negative',
 'neutral',
 'neutral',
 'neutral',
 'negative']

In [219]:
tweets_en['Sentiment'] = sentiment_category_en

In [220]:
# tweets_en.head()

In [221]:
sentiment_category_de = ['positive' if score >0
                         else 'negative' if score<0
                             else 'neutral'
                                  for score in tweets_de['Polarity']]

In [222]:
sentiment_category_de[:20] 
# Make sure it worked, then add to df

['negative',
 'positive',
 'neutral',
 'neutral',
 'positive',
 'positive',
 'negative',
 'positive',
 'neutral',
 'positive',
 'neutral',
 'positive',
 'neutral',
 'neutral',
 'neutral',
 'positive',
 'neutral',
 'neutral',
 'negative',
 'neutral']

In [223]:
tweets_de['Sentiment'] = sentiment_category_de

In [224]:
tweets_de.head()

,TweetText,Handle,Location,Country,Gender,Language,text_processed,Polarity,Subjectivity,Sentiment
0,RT @z_edian: Unser @snv_berlin Expert:innenwor...,christinacrupp,"Bonn, Deutschland",Germany,female,de,"[snvberlin, expertinnenworkshop, staatlich, be...",-0.350000,0.00,negative
2,RT @michael_adler_: Fing gut an! Bonn braucht ...,joscchh,"Bonn, Deutschland",Germany,female,de,"[fing, gut, bonn, braucht, mindshift, autostad...",0.250000,0.25,positive
4,"Gestern haben @Lichtemomente, @askans, @michae...",joscchh,"Bonn, Deutschland",Germany,female,de,"[gest, lichtemoment, askan, michaeladl, barbar...",0.000000,0.00,neutral
5,@VolkerGoebbels Macht Sinn 😏,joscchh,"Bonn, Deutschland",Germany,female,de,"[volkergoebbel, macht, sinn]",0.000000,0.00,neutral
6,@jbrunotte Was waren denn deine Beweggründe da...,joscchh,"Bonn, Deutschland",Germany,female,de,"[jbrunott, wa, beweggrund, dafur, spannend, wiss]",0.233333,0.00,positive


# Step 3:
### Putting the en/de tweets back into one dataframe for continued analysis:

In [225]:
print(len(tweets_en))
print(len(tweets_de))

# Save both en/de dataframes to csv for wordcloud in separate analysis:
tweets_en.to_csv('tweets_en.csv')
tweets_de.to_csv('tweets_de.csv')

4791
3070


In [229]:
tweets_analysis = pd.concat([tweets_en, tweets_de], ignore_index=True)

In [230]:
# save df to csv:

tweets_analysis.to_csv('tweets_analysis.csv')

In [231]:
tweets_analysis['Country'].value_counts()

Germany    5662
USA        2199
Name: Country, dtype: int64

In [232]:
type(tweets_analysis['text_processed'][0][1])

str

In [233]:
tweets_analysis['text_processed']

0       [daili, goal, wake, earli, drink, coffe, work,...
1                               [good, topic, well, done]
2       [jpbrice, life, alway, precious, everi, singl,...
3       [petershepherd, myork, zereadyoregon, great, l...
4       [never, know, strong, strong, onli, choic, quo...
                              ...                        
7856    [friedrich, merz, frau, geheiratet, mehr, femi...
7857    [hatt, nie, gedacht, mal, sag, druck, arminlas...
7858    [echt, frau, job, abgibt, au, grund, einfach, ...
7859    [perfid, begriff, neidsteu, uberhaupt, ausdenk...
7860    [gerad, foto, gepostet, berlin, germany, https...
Name: text_processed, Length: 7861, dtype: object

In [234]:
#### Since absolute numbers aren't telling, maybe look at ratio of positive:total tweets.

In [235]:
# Get length of German-region tweets for ratio:

ger = tweets_analysis[tweets_analysis['Country']=='Germany']
print("Tweets from German Network: ", len(ger))

usa = tweets_analysis[tweets_analysis['Country']=='USA']
print("Tweets from American Network: ", len(usa))

Tweets from German Network:  5662
Tweets from American Network:  2199


In [236]:
ratio_us_pos = len(tweets_analysis[(tweets_analysis['Country']=='USA') & (tweets_analysis['Sentiment']=='positive')])/len(usa)
ratio_us_pos

0.4492951341518872

In [237]:
ratio_de_pos = len(tweets_analysis[(tweets_analysis['Country']=='Germany') & (tweets_analysis['Sentiment']=='positive')])/len(ger)
ratio_de_pos

0.36276933945602263

# Step 4:
### Continue on here if time?

# RETURN TO HERE:

Although the graphs of all tweets make it seem that German Network is more positive, the ratio of positive tweets: all tweets in the US is 10% higher!

Get the predict_proba probability from TextBlob next!

In [238]:
# Try Groupby country:
tweets_analysis.groupby(['Country']).mean()

# Overall, while average tweets tend to be neutral, American tweets tend to be more SUBJECTIVE. 

,Polarity,Subjectivity
Country,,
Germany,0.121243,0.183735
USA,0.130116,0.328719


In [239]:
grouped_country = tweets_analysis.groupby(['Country', 'Sentiment']).mean()
grouped_country

# Overall, American tweets tend to be more SUBJECTIVE, in both positive and negative tweets. 

Polarity  Subjectivity
Country Sentiment                        
Germany negative  -0.325036      0.282079
        neutral    0.000000      0.060850
        positive   0.423306      0.338951
USA     negative  -0.250050      0.472570
        neutral    0.000000      0.075839
        positive   0.359453      0.527850

In [242]:
grouped_gender = tweets_analysis.groupby(['Gender', 'Sentiment']).mean()
grouped_gender

Polarity  Subjectivity
Gender  Sentiment                        
female  negative  -0.286795      0.368513
        neutral    0.000000      0.055351
        positive   0.389999      0.441795
male    negative  -0.324953      0.320576
        neutral    0.000000      0.073991
        positive   0.414592      0.364161
unknown negative  -0.244688      0.344207
        neutral    0.000000      0.059023
        positive   0.409246      0.363201